In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
17464789/17464789 [==============================] - 0s 0us/step


In [4]:
# Lets look at one review
len(train_data[1]), len(train_data[0])

(189, 218)

###More Preprocessing
We cannot pass different length data into our neural network

In [5]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

###Creating the Model
We'll use a word embedding layer as the first layer in our model and add a LSTM layer afterwards that feeds into a dense node to get our predicted sentiment.

32 stands for the output dimension of the vectors generated by the embedding layer. We can change this value if we'd like!

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid") # squishes the values between 0 and 1
])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


###Training

In [9]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 70s 109ms/step - loss: 0.4562 - acc: 0.7831 - val_loss: 0.2919 - val_acc: 0.8824
Epoch 2/10
625/625 [==============================] - 65s 104ms/step - loss: 0.2597 - acc: 0.8988 - val_loss: 0.4492 - val_acc: 0.8368
Epoch 3/10
625/625 [==============================] - 67s 107ms/step - loss: 0.1991 - acc: 0.9254 - val_loss: 0.2914 - val_acc: 0.8868
Epoch 4/10
625/625 [==============================] - 66s 105ms/step - loss: 0.1603 - acc: 0.9423 - val_loss: 0.3614 - val_acc: 0.8548
Epoch 5/10
625/625 [==============================] - 68s 109ms/step - loss: 0.1301 - acc: 0.9552 - val_loss: 0.3108 - val_acc: 0.8792
Epoch 6/10
625/625 [==============================] - 66s 106ms/step - loss: 0.1100 - acc: 0.9633 - val_loss: 0.3114 - val_acc: 0.8892
Epoch 7/10
625/625 [==============================] - 65s 104ms/step - loss: 0.0911 - acc: 0.9704 - val_loss: 0.3393 - val_acc: 0.8838
Epoch 8/10
625/625 [==============================] - 6

And we'll evaluate the model on our training data to see how well it performs.

In [10]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 19s 25ms/step - loss: 0.4745 - acc: 0.8622
[0.4744901657104492, 0.8621600270271301]


###Making Predictions
Now let’s use our network to make predictions on our own reviews.

Since our reviews are encoded well need to convert any review that we write into that form so the network can understand it. To do that well load the encodings from the dataset and use them to encode our own data.




In [14]:
word_index = imdb.get_word_index()
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)



[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4

Decode function

In [22]:

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]

print(decode_integers(encoded))

that movie was just amazing so amazing


In [31]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


1/1 [==============================] - 0s 88ms/step
[0.9596702]
1/1 [==============================] - 0s 77ms/step
[0.30503]
